In [1]:
#導入模型
from bs4 import BeautifulSoup
import requests
import json

In [2]:
# Specify the webpage for scraping
# bitcoin:https://www.ptt.cc/bbs/Gossiping/search?q=%E6%AF%94%E7%89%B9%E5%B9%A3
# cryptocurrency:https://www.ptt.cc/bbs/Gossiping/search?q=%E5%8A%A0%E5%AF%86%E8%B2%A8%E5%B9%A3

url = "https://www.ptt.cc/bbs/Gossiping/search?q=%E5%8A%A0%E5%AF%86%E8%B2%A8%E5%B9%A3"

In [3]:
payload = {
    'from': '/bbs/Gossiping/index.html',
    'yes': 'yes'
}
data = []   # 全部文章的資料
num = 0

In [4]:
# 因為有PTT有18禁問題，所以需要讓爬蟲自己按成年
# 用session紀錄此次使用的cookie
rs = requests.session()
response = rs.post("https://www.ptt.cc/ask/over18", data=payload)

In [5]:
# 爬取2頁試試看
for i in range(2):
    # get取得頁面的HTML
    # print(url)
    response = rs.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # print(soup.prettify())
    # 找出每篇文章的連結
    links = soup.find_all("div", class_="title")
    # print(links)
    for link in links:
        # 如果文章已被刪除，連結為None
        if link.a != None:
            article_data = {}   # 單篇文章的資料
            page_url = "https://www.ptt.cc/"+link.a["href"]
            # print(page_url)#這是會秀出來每篇文章網址
            
            # 進入文章頁面
            response = rs.get(page_url)
            result = BeautifulSoup(response.text, "html.parser")
            # print(soup.prettify())
            
            # 找出作者、標題、時間、留言
            main_content = result.find("div", id="main-content")
            article_info = main_content.find_all(
                "span", class_="article-meta-value")
            # print(article_info)
            
            if len(article_info) != 0:
                author = article_info[0].string  # 作者
                # kanban = article_info[1].string #看板
                title = article_info[2].string  # 標題
                time = article_info[3].string   # 時間
            else:
                author = "無"  # 作者
                title = "無"  # 標題
                time = "無"   # 時間

#             print(author)
#             print(title)
#             print(time)
            article_data["author"] = author
            article_data["title"] = title
            article_data["time"] = time
            
            # 將整段文字內容抓出來
            all_text = main_content.text
            # print(all_text)
            # 以--切割，抓最後一個--前的所有內容
            pre_texts = all_text.split("--")[:-1]
            # 將前面的所有內容合併成一個
            one_text = "--".join(pre_texts)
            # 以\n切割，第一行標題不要
            texts = one_text.split("\n")[1:]
            # 將每一行合併
            content = "\n".join(texts)
            # print(content)
            
            article_data["content"] = content

            # 一種留言一個列表
            comment_dic = {}
            push_dic = []
            arrow_dic = []
            shu_dic = []
            
            # 抓出所有留言
            comments = main_content.find_all("div", class_="push")
            for comment in comments:
                 # 檢查是否為warning-box
                if comment.find("div", class_="push center warning-box"):
                    # 如果是warning-box，跳過該留言並繼續下一個留言
                    continue
                push_tag = comment.find(
                    "span", class_="push-tag")   # 分類標籤
                push_tag = push_tag.string if push_tag  else ""
                
                push_userid = comment.find(
                    "span", class_="push-userid")  # 使用者ID
                push_userid = push_userid.string if push_userid else ""
                
                push_content = comment.find(
                    "span", class_="push-content")   # 留言內容
                push_content = push_content.string if push_content else ""
                
                push_time = comment.find(
                    "span", class_="push-ipdatetime")   # 留言時間
                push_time = push_time.string if push_time else ""
                
                if not push_content:
                    push_content = comment.select_one("span.push-content").text if comment.select_one("span.push-content") else ""
                    push_time = comment.find("span", class_="push-ipdatetime")
                    push_time = push_time.string if push_time else ""

                
                # 檢查是否有richcontent-圖片
                richcontent = comment.find("div", class_="richcontent")
                if richcontent:
                    # 將richcontent轉換為字串並加入留言內容
                    push_content += richcontent.text
                # print(push_tag, push_userid, push_content, push_time)
                
                
                dict1 = {"push_userid": push_userid,
                         "push_content": push_content, "push_time": push_time}
                # dict1 = {"push_content": push_content, "push_time": push_time}

                if push_tag == "推 ":
                         push_dic.append(dict1)
                if push_tag == "→ ":
                         arrow_dic.append(dict1)
                if push_tag == "噓 ":
                         shu_dic.append(dict1)

            # print(push_dic)
            # print(arrow_dic)
            # print(shu_dic)
            # print("--------")

            comment_dic["推"] = push_dic
            comment_dic["→"] = arrow_dic
            comment_dic["噓"] = shu_dic
            article_data["comment"] = comment_dic

            # print(article_data)
            data.append(article_data)
            num += 1
            print("第 "+str(num)+" 篇文章完成!")

    # 找到上頁的網址，並更新url
    url = "https://www.ptt.cc/"+soup.find("a", string="‹ 上頁")["href"]


# print(data)
# 輸出JSON檔案
with open('./PTT_data/Original_Data/Original_crypto_Data_Ptt_job.json', 'w', encoding='utf-8') as f:
    json.dump(data, f)

第 1 篇文章完成!
第 2 篇文章完成!
第 3 篇文章完成!
第 4 篇文章完成!
第 5 篇文章完成!
第 6 篇文章完成!
第 7 篇文章完成!
第 8 篇文章完成!
第 9 篇文章完成!
第 10 篇文章完成!
第 11 篇文章完成!
第 12 篇文章完成!
第 13 篇文章完成!
第 14 篇文章完成!
第 15 篇文章完成!
第 16 篇文章完成!
第 17 篇文章完成!
第 18 篇文章完成!
第 19 篇文章完成!
第 20 篇文章完成!
第 21 篇文章完成!
第 22 篇文章完成!
第 23 篇文章完成!
第 24 篇文章完成!
第 25 篇文章完成!
第 26 篇文章完成!
第 27 篇文章完成!
第 28 篇文章完成!
第 29 篇文章完成!
第 30 篇文章完成!
第 31 篇文章完成!
第 32 篇文章完成!
第 33 篇文章完成!
第 34 篇文章完成!
第 35 篇文章完成!
第 36 篇文章完成!
第 37 篇文章完成!
第 38 篇文章完成!
第 39 篇文章完成!
第 40 篇文章完成!
